## TABLE LOOKUP METHOD OF Q LEARNING

In [ ]:
import numpy as np
import gym
import matplotlib.pyplot as plt

# Import and initialize Mountain Car Environment
env = gym.make('MountainCar-v0')
env.reset()

# Define Q-learning function
def QLearning(env, learning, discount, epsilon, min_eps, episodes):
    # Determine size of discretized state space
    num_states = (env.observation_space.high - env.observation_space.low)*\
                    np.array([10, 100])
    num_states = np.round(num_states, 0).astype(int) + 1
    
    # Initialize Q table
    Q = np.random.uniform(low = -1, high = 1, 
                          size = (num_states[0], num_states[1], 
                                  env.action_space.n))
    
    # Initialize variables to track rewards
    reward_list = []
    ave_reward_list = []
    
    # Calculate episodic reduction in epsilon
    reduction = (epsilon - min_eps)/episodes
    
    # Run Q learning algorithm
    for i in range(episodes):
        # Initialize parameters
        done = False
        tot_reward, reward = 0,0
        state = env.reset()
        
        # Discretize state
        state_adj = (state - env.observation_space.low)*np.array([10, 100])
        state_adj = np.round(state_adj, 0).astype(int)
    
        while done != True:   
            # Render environment for last five episodes
            if i >= (episodes - 20):
                env.render()
                
            # Determine next action - epsilon greedy strategy
            if np.random.random() < 1 - epsilon:
                action = np.argmax(Q[state_adj[0], state_adj[1]]) 
            else:
                action = np.random.randint(0, env.action_space.n)
                
            # Get next state and reward
            state2, reward, done, info = env.step(action) 
            
            # Discretize state2
            state2_adj = (state2 - env.observation_space.low)*np.array([10, 100])
            state2_adj = np.round(state2_adj, 0).astype(int)
            
            #Allow for terminal states
            if done and state2[0] >= 0.5:
                Q[state_adj[0], state_adj[1], action] = reward
                
            # Adjust Q value for current state
            else:
                delta = learning*(reward + 
                                 discount*np.max(Q[state2_adj[0], 
                                                   state2_adj[1]]) - 
                                 Q[state_adj[0], state_adj[1],action])
                Q[state_adj[0], state_adj[1],action] += delta
                                     
            # Update variables
            tot_reward += reward
            state_adj = state2_adj
        
        # Decay epsilon
        if epsilon > min_eps:
            epsilon -= reduction
        
        # Track rewards
        reward_list.append(tot_reward)
        
        if (i+1) % 100 == 0:
            ave_reward = np.mean(reward_list)
            ave_reward_list.append(ave_reward)
            reward_list = []
            
        if (i+1) % 100 == 0:    
            print('Episode {} Average Reward: {}'.format(i+1, ave_reward))
            
    env.close()
    
    return ave_reward_list

# Run Q-learning algorithm
rewards = QLearning(env, 0.2, 0.9, 0.8, 0, 5000)

# Plot Rewards
plt.plot(100*(np.arange(len(rewards)) + 1), rewards)
plt.xlabel('Episodes')
plt.ylabel('Average Reward')
plt.title('Average Reward vs Episodes')
#plt.savefig('rewards.jpg')     
plt.close() 

## NN METHOD OF Q LEARNING WITH EXPERIENCE REPLAY

In [6]:
import numpy as np
import collections
from collections import deque
import tensorflow as tf
import tqdm
from tqdm import tqdm
def dense(x, weights, bias, activation=tf.identity):
    """Dense layer."""
    z = tf.matmul(x, weights) + bias
    return activation(z)


def init_weights(shape, initializer):
    """Initialize weights for tensorflow layer."""
    weights = tf.Variable(
        initializer(shape),
        trainable=True,
        dtype=tf.float32
    )

    return weights


class Network(object):
    """Q-function approximator."""

    def __init__(self,
                 input_size,
                 output_size,
                 hidden_size=[50, 50],
                 weights_initializer=tf.initializers.glorot_uniform(),
                 bias_initializer=tf.initializers.zeros(),
                 optimizer=tf.optimizers.Adam,
                 **optimizer_kwargs):
        """Initialize weights and hyperparameters."""
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size

        np.random.seed(41)

        self.initialize_weights(weights_initializer, bias_initializer)
        self.optimizer = optimizer(**optimizer_kwargs)
        self.update_count = 0 # this will tell how many times the train_step method is executed
        
    def initialize_weights(self, weights_initializer, bias_initializer):
        """Initialize and store weights."""
        wshapes = [
            [self.input_size, self.hidden_size[0]],
            [self.hidden_size[0], self.hidden_size[1]],
            [self.hidden_size[1], self.output_size]
        ]

        bshapes = [
            [1, self.hidden_size[0]],
            [1, self.hidden_size[1]],
            [1, self.output_size]
        ]

        self.weights = [init_weights(s, weights_initializer) for s in wshapes]
        self.biases = [init_weights(s, bias_initializer) for s in bshapes]

        self.trainable_variables = self.weights + self.biases

    def model(self, inputs):
        """Given a state vector, return the Q values of actions."""
        h1 = dense(inputs, self.weights[0], self.biases[0], tf.nn.relu)
        h2 = dense(h1, self.weights[1], self.biases[1], tf.nn.relu)

        out = dense(h2, self.weights[2], self.biases[2])

        return out

    def train_step(self, inputs, targets, actions_one_hot):
        """Update weights."""
        with tf.GradientTape() as tape:
            qvalues = tf.squeeze(self.model(inputs))
            preds = tf.reduce_sum(qvalues * actions_one_hot, axis=1)
            loss = tf.losses.mean_squared_error(targets, preds)

        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        self.update_count = self.update_count + 1
        #print("NN_updated : ",self.update_count + 1)

class Memory(object):
    """Memory buffer for Experience Replay."""

    def __init__(self, max_size):
        """Initialize a buffer containing max_size experiences."""
        self.buffer = deque(maxlen=max_size)

    def add(self, experience):
        """Add an experience to the buffer."""
        self.buffer.append(experience)

    def sample(self, batch_size):
        """Sample a batch of experiences from the buffer."""
        buffer_size = len(self.buffer)
        index = np.random.choice(
            np.arange(buffer_size),
            size=batch_size,
            replace=False
        )

        return [self.buffer[i] for i in index]

    def __len__(self):
        """Interface to access buffer length."""
        return len(self.buffer)


class Agent(object):
    """Deep Q-learning agent."""

    def __init__(self,
                 state_space_size,
                 action_space_size,
                 target_update_freq=1000,
                 discount=0.99,
                 batch_size=32,
                 max_explore=1,
                 min_explore=0.05,
                 anneal_rate=(1 / 100000),
                 replay_memory_size=100000,
                 replay_start_size=10000):
        """Set parameters, initialize network."""
        self.action_space_size = action_space_size
        # DQN has 2 networks THESE ARE STATE (V) ESTIMATORS AND NOT Q ESTIMATORS
        # hell this is a policy gradient method it is not even a V estimator
        self.online_network = Network(state_space_size, action_space_size)
        self.target_network = Network(state_space_size, action_space_size)

        self.update_target_network()

        # training parameters
        self.target_update_freq = target_update_freq
        self.discount = discount
        self.batch_size = batch_size

        # policy during learning
        self.max_explore = max_explore + (anneal_rate * replay_start_size)
        self.min_explore = min_explore
        self.anneal_rate = anneal_rate
        self.steps = 0

        # replay memory
        self.memory = Memory(replay_memory_size)
        self.replay_start_size = replay_start_size
        self.experience_replay = Memory(replay_memory_size)
        
        self.last_state = []
        self.last_action = None
        
    def handle_episode_start(self):
        self.last_state, self.last_action = [], None

    def step(self, observation, training=True):
        """Observe state and rewards, select action.

        It is assumed that `observation` will be an object with
        a `state` vector and a `reward` float or integer. The reward
        corresponds to the action taken in the previous step.
        """
        last_state, last_action = self.last_state, self.last_action
        last_reward = observation[1]
        state = observation[0]
        
        action = self.policy(state, training)

        if training:
            self.steps += 1

            if len(last_state) != 0:
                experience = {
                    "state": last_state,
                    "action": last_action,
                    "reward": last_reward,
                    "next_state": state
                }

                self.memory.add(experience)

            if self.steps > self.replay_start_size:
                self.train_network()

                if self.steps % self.target_update_freq == 0:
                    self.update_target_network()

        self.last_state = state
        self.last_action = action
        #print("Step Taken ", self.steps)
        return action
    
    def policy(self,state, training):
        """Epsilon-greedy policy for training, greedy policy otherwise."""
        explore_prob = self.max_explore - (self.steps * self.anneal_rate)
        explore = max(explore_prob, self.min_explore) > np.random.rand()

        if training and explore:
            action = np.random.randint(self.action_space_size)
        else:
            inputs = np.expand_dims(state, 0)
            qvalues = self.online_network.model(inputs)
            action = np.squeeze(np.argmax(qvalues, axis=-1))

        return action

    def update_target_network(self):
        """Update target network weights with current online network values."""
        variables = self.online_network.trainable_variables
        variables_copy = [tf.Variable(v) for v in variables]
        self.target_network.trainable_variables = variables_copy

    def train_network(self):
        """Update online network weights."""
        batch = self.memory.sample(self.batch_size)
        inputs = np.array([b["state"] for b in batch],dtype = 'float32')
        actions = np.array([b["action"] for b in batch])
        rewards = np.array([b["reward"] for b in batch],dtype = 'float32')
        next_inputs = np.array([b["next_state"] for b in batch],dtype = 'float32')
        
        actions_one_hot = np.eye(self.action_space_size)[actions]
        actions_one_hot = np.array(actions_one_hot,dtype = 'float32')
        
        next_qvalues = np.squeeze(self.target_network.model(next_inputs))
        targets = rewards + self.discount * np.amax(next_qvalues, axis=-1)
        self.online_network.train_step(inputs, targets, actions_one_hot)

In [7]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [8]:
import numpy as np
import gym
import matplotlib.pyplot as plt

# starting the environment
env = gym.make('MountainCar-v0')
observation_space = 2
actions = 3

In [9]:
# initializing the agent

agent47 = Agent(
state_space_size = observation_space,
action_space_size = actions
)

In [10]:
# training loop

for i in tqdm(range(1000)):
    print(i)
    state = env.reset()
    state = np.array(state,dtype = 'float32')
    last_rew = 0
    last_rew = np.array(last_rew,dtype = 'float32')
    done = False
    for j in range(1000):
        if not done:
            if i > 990:
                env.render()
            if j == 0:
                obs = np.array([state,last_rew])
            state,reward,done,info = env.step(agent47.step(obs,training = True))
            state = np.array(state,dtype = 'float32')
            reward = np.array(reward,dtype = 'float32')
            obs = np.array([state,reward])

        
env.close()

  2%|▏         | 20/1000 [00:00<00:10, 94.05it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28

  4%|▍         | 42/1000 [00:00<00:09, 97.79it/s]


29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


  5%|▌         | 51/1000 [00:01<00:51, 18.32it/s]

...

100%|█████████▉| 998/1000 [38:32<00:07,  3.72s/it]

998


100%|█████████▉| 999/1000 [38:36<00:03,  3.75s/it]

999


100%|██████████| 1000/1000 [38:40<00:00,  2.32s/it]


In [6]:
env.close()

In [10]:
#testing loop
act = []
for i in tqdm(range(10)):
    print(i)
    state = env.reset()
    state = np.array(state,dtype = 'float32')
    last_rew = 0
    last_rew = np.array(last_rew,dtype = 'float32')
    for j in range(1000):
        env.render()
        if j == 0:
            obs = np.array([state,last_rew])
        action = agent47.step(obs,training = False)
        act.append(action)
        state,reward,done,info = env.step(action)
        state = np.array(state,dtype = 'float32')
        reward = np.array(reward,dtype = 'float32')
        obs = np.array([state,reward])

        
env.close()

  0%|          | 0/10 [00:00<?, ?it/s]

0


  0%|          | 0/10 [00:06<?, ?it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\ishtd\Anaconda3\envs\deeplearning\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-0ffed353310c>", line 10, in <module>
    env.render()
  File "C:\Users\ishtd\Anaconda3\envs\deeplearning\lib\site-packages\gym\core.py", line 233, in render
    return self.env.render(mode, **kwargs)
  File "C:\Users\ishtd\Anaconda3\envs\deeplearning\lib\site-packages\gym\envs\classic_control\mountain_car.py", line 119, in render
    return self.viewer.render(return_rgb_array = mode=='rgb_array')
  File "C:\Users\ishtd\Anaconda3\envs\deeplearning\lib\site-packages\gym\envs\classic_control\rendering.py", line 124, in render
    self.window.flip()
  File "C:\Users\ishtd\Anaconda3\envs\deeplearning\lib\site-packages\pyglet\window\win32\__init__.py", line 336, in flip
    _dwmapi.DwmFlush()
KeyboardInterrupt

During handling of the above exception, ano

TypeError: object of type 'NoneType' has no len()

In [11]:
env.close()

ERROR! Session/line number was not unique in database. History logging moved to new session 753


In [12]:
act

[array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=int64),
 array(0, dtype=

In [15]:
for i in act:
    print(i)

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


In [16]:
len(act)

3128

## KERAS RL

In [13]:
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents import SARSAAgent
from rl.policy import BoltzmannQPolicy


Using TensorFlow backend.


In [19]:
ENV_NAME = 'MountainCar-v0'

# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

# Next, we build a very simple model.
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 2)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)                48        
_________________________________________________________________
activation_5 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_6 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_7 (Activation)    (None, 16)               

In [20]:
# SARSA does not require a memory.
policy = BoltzmannQPolicy()
sarsa = SARSAAgent(model=model, nb_actions=nb_actions, nb_steps_warmup=10, policy=policy)
sarsa.compile(Adam(lr=1e-3), metrics=['mae'])

In [22]:
sarsa.fit(env, nb_steps=50000, visualize=False, verbose=2)

Training for 50000 steps ...
   200/50000: episode: 1, duration: 1.589s, episode steps: 200, steps per second: 126, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.155 [0.000, 2.000], mean observation: -0.254 [-0.584, 0.008], loss: 22.363530, mae: 54.889876, mean_q: -81.644252
   400/50000: episode: 2, duration: 1.641s, episode steps: 200, steps per second: 122, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.285 [0.000, 2.000], mean observation: -0.245 [-0.650, 0.020], loss: 22.040751, mae: 57.202113, mean_q: -84.224867
   600/50000: episode: 3, duration: 1.653s, episode steps: 200, steps per second: 121, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.360 [0.000, 2.000], mean observation: -0.235 [-0.626, 0.017], loss: 19.601438, mae: 53.201671, mean_q: -78.640333
   800/50000: episode: 4, duration: 1.652s, episode steps: 200, steps per second: 121, episode reward: -200.000, mean reward: -1.00

  6200/50000: episode: 31, duration: 1.651s, episode steps: 200, steps per second: 121, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.395 [0.000, 2.000], mean observation: -0.233 [-0.567, 0.008], loss: 12.536173, mae: 43.032049, mean_q: -63.197145
  6400/50000: episode: 32, duration: 1.637s, episode steps: 200, steps per second: 122, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.685 [0.000, 2.000], mean observation: -0.289 [-0.901, 0.027], loss: 17.831770, mae: 46.627816, mean_q: -68.536005
  6600/50000: episode: 33, duration: 1.627s, episode steps: 200, steps per second: 123, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.085 [0.000, 2.000], mean observation: -0.254 [-0.565, 0.008], loss: 9.281157, mae: 43.990830, mean_q: -64.109612
  6800/50000: episode: 34, duration: 1.680s, episode steps: 200, steps per second: 119, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean a

 12200/50000: episode: 61, duration: 1.700s, episode steps: 200, steps per second: 118, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.840 [0.000, 2.000], mean observation: -0.279 [-0.842, 0.025], loss: 16.090722, mae: 45.989771, mean_q: -67.775025
 12400/50000: episode: 62, duration: 1.682s, episode steps: 200, steps per second: 119, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.675 [0.000, 2.000], mean observation: -0.290 [-0.787, 0.015], loss: 13.983935, mae: 46.208318, mean_q: -67.724652
 12600/50000: episode: 63, duration: 1.688s, episode steps: 200, steps per second: 119, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.100 [0.000, 2.000], mean observation: -0.256 [-0.612, 0.008], loss: 10.133814, mae: 43.539270, mean_q: -64.196401
 12800/50000: episode: 64, duration: 1.646s, episode steps: 200, steps per second: 122, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean 

 18200/50000: episode: 91, duration: 1.698s, episode steps: 200, steps per second: 118, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.405 [0.000, 2.000], mean observation: -0.233 [-0.587, 0.008], loss: 10.855528, mae: 43.390440, mean_q: -63.658402
 18400/50000: episode: 92, duration: 1.745s, episode steps: 200, steps per second: 115, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.860 [0.000, 2.000], mean observation: -0.278 [-0.817, 0.024], loss: 14.494032, mae: 46.160267, mean_q: -68.018954
 18600/50000: episode: 93, duration: 1.797s, episode steps: 200, steps per second: 111, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.030 [0.000, 2.000], mean observation: -0.260 [-0.615, 0.009], loss: 11.467219, mae: 44.753378, mean_q: -65.762755
 18800/50000: episode: 94, duration: 1.825s, episode steps: 200, steps per second: 110, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean 

 24200/50000: episode: 121, duration: 1.622s, episode steps: 200, steps per second: 123, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.990 [0.000, 2.000], mean observation: -0.258 [-0.602, 0.010], loss: 12.803362, mae: 45.066047, mean_q: -66.347507
 24400/50000: episode: 122, duration: 1.675s, episode steps: 200, steps per second: 119, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.900 [0.000, 2.000], mean observation: -0.271 [-0.727, 0.015], loss: 14.123809, mae: 45.752264, mean_q: -67.359591
 24600/50000: episode: 123, duration: 1.663s, episode steps: 200, steps per second: 120, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.310 [0.000, 2.000], mean observation: -0.236 [-0.654, 0.017], loss: 10.172039, mae: 43.363581, mean_q: -63.748072
 24800/50000: episode: 124, duration: 1.660s, episode steps: 200, steps per second: 121, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], m

 30200/50000: episode: 151, duration: 1.649s, episode steps: 200, steps per second: 121, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.450 [0.000, 2.000], mean observation: -0.226 [-0.588, 0.013], loss: 12.341633, mae: 43.064786, mean_q: -63.402677
 30400/50000: episode: 152, duration: 1.685s, episode steps: 200, steps per second: 119, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.420 [0.000, 2.000], mean observation: -0.229 [-0.623, 0.011], loss: 10.673128, mae: 43.434006, mean_q: -63.690645
 30600/50000: episode: 153, duration: 1.686s, episode steps: 200, steps per second: 119, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.320 [0.000, 2.000], mean observation: -0.234 [-0.733, 0.021], loss: 8.928185, mae: 43.599091, mean_q: -64.281562
 30800/50000: episode: 154, duration: 1.661s, episode steps: 200, steps per second: 120, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], me

 36200/50000: episode: 181, duration: 1.661s, episode steps: 200, steps per second: 120, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.950 [0.000, 2.000], mean observation: -0.272 [-0.751, 0.019], loss: 14.044078, mae: 45.256840, mean_q: -66.912602
 36400/50000: episode: 182, duration: 1.652s, episode steps: 200, steps per second: 121, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.485 [0.000, 2.000], mean observation: -0.225 [-0.592, 0.011], loss: 11.729991, mae: 42.676716, mean_q: -62.369840
 36600/50000: episode: 183, duration: 1.700s, episode steps: 200, steps per second: 118, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.040 [0.000, 2.000], mean observation: -0.264 [-0.701, 0.012], loss: 13.060466, mae: 44.952912, mean_q: -66.405325
 36800/50000: episode: 184, duration: 1.753s, episode steps: 200, steps per second: 114, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], m

 42200/50000: episode: 211, duration: 1.652s, episode steps: 200, steps per second: 121, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.885 [0.000, 2.000], mean observation: -0.276 [-0.776, 0.021], loss: 12.032395, mae: 45.565283, mean_q: -67.164056
 42400/50000: episode: 212, duration: 1.650s, episode steps: 200, steps per second: 121, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.190 [0.000, 2.000], mean observation: -0.250 [-0.614, 0.005], loss: 12.948334, mae: 44.056827, mean_q: -65.036085
 42600/50000: episode: 213, duration: 1.641s, episode steps: 200, steps per second: 122, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.355 [0.000, 2.000], mean observation: -0.236 [-0.571, 0.006], loss: 10.862045, mae: 43.367203, mean_q: -63.645797
 42800/50000: episode: 214, duration: 1.650s, episode steps: 200, steps per second: 121, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], m

 48200/50000: episode: 241, duration: 1.642s, episode steps: 200, steps per second: 122, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.410 [0.000, 2.000], mean observation: -0.234 [-0.661, 0.014], loss: 9.280254, mae: 43.534326, mean_q: -63.963901
 48400/50000: episode: 242, duration: 1.652s, episode steps: 200, steps per second: 121, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.890 [0.000, 2.000], mean observation: -0.271 [-0.692, 0.017], loss: 13.916360, mae: 45.471722, mean_q: -67.316790
 48600/50000: episode: 243, duration: 1.670s, episode steps: 200, steps per second: 120, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.810 [0.000, 2.000], mean observation: -0.274 [-0.699, 0.014], loss: 14.443666, mae: 45.227689, mean_q: -66.780255
 48800/50000: episode: 244, duration: 1.678s, episode steps: 200, steps per second: 119, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], me

In [23]:
# Finally, evaluate our algorithm for 5 episodes.
sarsa.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: -200.000, steps: 200
Episode 2: reward: -200.000, steps: 200
Episode 3: reward: -200.000, steps: 200
Episode 4: reward: -200.000, steps: 200
Episode 5: reward: -200.000, steps: 200


In [24]:
env.close()